In [2]:
import matplotlib.pyplot as plt
import numpy as np
import xarray as xr
import pandas as pd
import os
import seaborn as sns
import random 
import scipy as sc
import statsmodels.api as sm
import scipy.stats as stats
import cartopy.crs as ccrs
import pygrib
import cfgrib

sns.set_theme(style="white")
sns.set_style("white")

random.seed(10)

# Import data in pandas dataframe

In [3]:
# Load data from MED-R Preindustrial and increased, as before
directory_exp = '/network/group/aopp/predict/AWH012_LEACH_NASTORM/DATA/MED-R/EXP/'

experiments = ['incr', 'pi']
cfpf = ['cf', 'pf']  # control and perturbed are treated equally
inits = {'pi' : ['b2nq_2022-02-10', 'b2nn_2022-02-14', 'b2ns_2022-02-16'], 
           'incr' : ['b2nr_2022-02-10', 'b2no_2022-02-14', 'b2nt_2022-02-16']}  # members for incr and pi runs

In [4]:
# empty data frame to be filled later
south_df = pd.DataFrame({'lat': [], 
                         'lon' : [],  
                         'cfpf' : [], 
                         'member' : [], 
                         'init' : [],
                         'time' : [],
                         'fg10' : []})

In [5]:
# Import lat, lon values
experiment = 'pi'
init = inits['pi'][0]
cont = 'cf'

lat = xr.open_dataset(os.path.join(directory_exp,experiment,'EU025/sfc',cont,init+'.nc')).latitude.values
lon = xr.open_dataset(os.path.join(directory_exp,experiment,'EU025/sfc',cont,init+'.nc')).longitude.values
 
# Import time steps, NB there are three sets of time steps depending on the initialisation date
fridays = []
for init in inits['pi']:  # time stamps are the same across experiments pi and icnr
    time = xr.open_dataset(os.path.join(directory_exp,experiment,'EU025/sfc',cont,init+'.nc')).time.values
    fridays.append((time > pd.Timestamp(2022,2,18, 0)) & (time < pd.Timestamp(2022,2,18, 18)))

# Defining box to analyse winds, south england and wales
lat1 = 52.2
lat2 = 50.3
lon1 = -6
lon2 = 1.3

# Create mask
south_england = {'lat': (lat < lat1) & (lat > lat2), 'lon': (lon < lon2) & (lon > lon1)}

# Import time steps, NB there are three sets of time steps depending on the initialisation date
fridays = []
for init in inits['pi']:  # time stamps are the same across experiments pi and icnr
    time = xr.open_dataset(os.path.join(directory_exp,experiment,'EU025/sfc',cont,init+'.nc')).time.values
    fridays.append((time > pd.Timestamp(2022,2,18, 0)) & (time < pd.Timestamp(2022,2,18, 18)))

# create a grid for lat and lon of the southern uk patch used above
llat, llon = np.meshgrid(lon, lat)

In [23]:
# Fill data frame
members = 50
for experiment in experiments:
    for init in inits[experiment]:
        for cont in cfpf:
            
            # import full data set in file
            data = xr.open_dataset(os.path.join(directory_exp,experiment,'EU025/sfc',cont,init+'.nc'))
            
            # store data in data frame with meta data
            if cont == 'cf':  # distinguish between cfpf because members in pf
                length = len(data.fg10.values.flatten())
                number_timesteps = len(data.time.values)
                adding = pd.DataFrame({'lat': np.repeat(llat.flatten(), number_timesteps), 
                                        'lon' : np.repeat(llon.flatten(), number_timesteps), 
                                        'cfpf' : np.tile(cont, length), 
                                        'member' : np.tile(-1, length), 
                                        'init' : np.tile(init, length), 
                                        'time': np.repeat(data.time.values.flatten(), len(lat)*len(lon)), 
                                        'fg10' : data.fg10.values.flatten()})
                south_df = pd.merge(south_df, adding,
                                    how = 'outer')
            
            elif cont == 'pf': 
                next
                # for member in range(members):
                #     length = len(data.fg10.values[:,member,:,:].flatten())
                #     number_timesteps = len(data.time.values)
                #     adding = pd.DataFrame({'lat': np.repeat(llat.flatten(), number_timesteps), 
                #                            'lon' : np.repeat(llon.flatten(), number_timesteps), 
                #                            'cfpf' : np.tile(cont, length),
                #                            'member' : np.tile(member, length), 
                #                            'init' : np.tile(init, length), 
                #                            'time': np.repeat(data.time.values.flatten(), len(lat)*len(lon)), 
                #                            'fg10' : data.fg10.values[:,member,:,:].flatten()})
                #     south_df = pd.merge(south_df, adding,
                #                         how = 'outer')